In [201]:
import pandas as pd 
from pathlib import Path
import numpy as np
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import matplotlib
import sqlite3

In [137]:
# ETL

In [164]:
# Read in the CSV file as a Pandas Dataframe
df = pd.read_csv(Path("resources/baseballData.csv"))
#ridding csv of partial seasons
dfFullYear = df[~df['year'].isin([2020, 2024])]
#dropping unesscessary columns
#'babip','bacon',
dfFullYear.drop(columns=["player_id",'player_age', "last_name, first_name", 'year', 'ab', 'pa', 'hit', 'single', 'double', 'triple', 'out_zone_swing_miss', 'babip','bacon',
       'out_zone_swing', 'out_zone_percent', 'out_zone', 'in_zone_swing_miss', 'single', 'double', 'triple', 'home_run', 'strikeout', 'walk', 'bb_percent', 'on_base_percent', 'on_base_plus_slg', 'isolated_power','woba','wobacon',
       'in_zone_swing','batted_ball', 'groundballs', 'flyballs', 'linedrives', 'popups',
       'home_run', 'strikeout', 'walk'], inplace=True)
dfFullYear.columns
dfFullYear.head(10)

/var/folders/q8/t_k5gtj11z5dpr8r51m6nbz00000gn/T/ipykernel_84724/1161540600.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfFullYear.drop(columns=["player_id",'player_age', "last_name, first_name", 'year', 'ab', 'pa', 'hit', 'single', 'double', 'triple', 'out_zone_swing_miss', 'babip','bacon',


,k_percent,batting_avg,slg_percent,exit_velocity_avg,launch_angle_avg,sweet_spot_percent,barrel,barrel_batted_rate,solidcontact_percent,flareburner_percent,...,whiff_percent,swing_percent,pull_percent,straightaway_percent,opposite_percent,f_strike_percent,groundballs_percent,flyballs_percent,linedrives_percent,popups_percent
0,18.5,0.240,0.409,88.5,10.8,28.5,21,5.0,5.2,21.6,...,23.1,53.4,34.7,40.6,24.5,64.0,49.4,22.6,20.9,7.1
1,15.5,0.273,0.553,93.0,15.7,34.8,58,13.1,8.4,24.9,...,23.2,44.7,41.9,36.4,21.7,52.1,37.6,25.6,28.7,8.1
2,23.4,0.250,0.486,91.3,12.2,31.4,42,10.9,8.1,20.3,...,32.0,43.9,38.4,43.9,16.9,60.3,43.6,24.9,24.9,6.5
3,13.2,0.246,0.423,87.4,15.8,33.5,23,5.6,6.6,23.3,...,17.9,52.9,36.9,38.3,24.8,65.5,37.6,24.5,29.1,8.7
4,10.5,0.287,0.453,89.5,12.6,35.7,28,5.5,7.8,26.3,...,16.8,48.1,35.3,41.4,23.3,61.9,42.4,18.4,31.6,7.6
5,16.0,0.276,0.471,90.6,15.6,34.1,23,5.8,7.5,27.3,...,18.1,45.4,40.6,38.3,19.3,57.3,36.8,26.6,28.1,8.5
6,15.3,0.224,0.358,85.7,16.2,32.1,11,2.5,4.2,23.8,...,18.1,45.0,46.0,32.6,21.2,56.5,41.3,24.7,24.2,9.7
7,10.9,0.244,0.480,90.7,14.2,29.5,41,7.7,5.3,22.7,...,16.3,45.4,43.3,37.0,19.5,56.0,42.8,25.5,23.8,7.9
8,26.7,0.247,0.453,86.3,12.1,33.9,31,8.5,5.5,21.9,...,31.2,60.2,36.6,35.8,27.6,70.4,43.7,22.7,28.7,4.9
9,16.8,0.265,0.380,89.5,4.4,33.3,18,3.7,8.0,28.6,...,16.8,40.7,28.2,39.7,32.1,57.2,57.6,13.4,27.2,1.9


In [165]:
dfFullYear.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1104 entries, 0 to 1245
Data columns (total 33 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   k_percent               1104 non-null   float64
 1   batting_avg             1104 non-null   float64
 2   slg_percent             1104 non-null   float64
 3   exit_velocity_avg       1104 non-null   float64
 4   launch_angle_avg        1104 non-null   float64
 5   sweet_spot_percent      1104 non-null   float64
 6   barrel                  1104 non-null   int64  
 7   barrel_batted_rate      1104 non-null   float64
 8   solidcontact_percent    1104 non-null   float64
 9   flareburner_percent     1104 non-null   float64
 10  poorlyunder_percent     1104 non-null   float64
 11  poorlytopped_percent    1104 non-null   float64
 12  poorlyweak_percent      1104 non-null   float64
 13  hard_hit_percent        1104 non-null   float64
 14  avg_best_speed          1104 non-null   float

In [166]:
dfFullYear.to_csv('Resources/Baseball_stats_cleaned.csv', index = False)

In [141]:
# SQL DB

In [167]:
# Step1. Load data file
df_cleaned = pd.read_csv('Resources/Baseball_stats_cleaned.csv')

In [168]:
df_cleaned.columns

Index(['k_percent', 'batting_avg', 'slg_percent', 'exit_velocity_avg',
       'launch_angle_avg', 'sweet_spot_percent', 'barrel',
       'barrel_batted_rate', 'solidcontact_percent', 'flareburner_percent',
       'poorlyunder_percent', 'poorlytopped_percent', 'poorlyweak_percent',
       'hard_hit_percent', 'avg_best_speed', 'avg_hyper_speed',
       'z_swing_percent', 'z_swing_miss_percent', 'oz_swing_percent',
       'oz_swing_miss_percent', 'oz_contact_percent', 'meatball_swing_percent',
       'iz_contact_percent', 'whiff_percent', 'swing_percent', 'pull_percent',
       'straightaway_percent', 'opposite_percent', 'f_strike_percent',
       'groundballs_percent', 'flyballs_percent', 'linedrives_percent',
       'popups_percent'],
      dtype='object')

In [169]:
# Step2. Create/connect to SQLite DB.
connection = sqlite3.connect('stats.db')

In [170]:
# Step3. Load dataset to SQLite DB
df_cleaned.to_sql('baseball_stats', connection, if_exists = 'replace')

1104

In [171]:
#Scale the Data
scaler = StandardScaler()
scaler.fit(dfFullYear)

StandardScaler()

In [172]:
df_cleaned


,k_percent,batting_avg,slg_percent,exit_velocity_avg,launch_angle_avg,sweet_spot_percent,barrel,barrel_batted_rate,solidcontact_percent,flareburner_percent,...,whiff_percent,swing_percent,pull_percent,straightaway_percent,opposite_percent,f_strike_percent,groundballs_percent,flyballs_percent,linedrives_percent,popups_percent
0,18.5,0.240,0.409,88.5,10.8,28.5,21,5.0,5.2,21.6,...,23.1,53.4,34.7,40.6,24.5,64.0,49.4,22.6,20.9,7.1
1,15.5,0.273,0.553,93.0,15.7,34.8,58,13.1,8.4,24.9,...,23.2,44.7,41.9,36.4,21.7,52.1,37.6,25.6,28.7,8.1
2,23.4,0.250,0.486,91.3,12.2,31.4,42,10.9,8.1,20.3,...,32.0,43.9,38.4,43.9,16.9,60.3,43.6,24.9,24.9,6.5
3,13.2,0.246,0.423,87.4,15.8,33.5,23,5.6,6.6,23.3,...,17.9,52.9,36.9,38.3,24.8,65.5,37.6,24.5,29.1,8.7
4,10.5,0.287,0.453,89.5,12.6,35.7,28,5.5,7.8,26.3,...,16.8,48.1,35.3,41.4,23.3,61.9,42.4,18.4,31.6,7.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1099,19.4,0.285,0.506,90.0,11.0,32.7,34,7.6,7.8,22.0,...,21.5,45.8,38.4,36.9,24.7,57.9,45.3,24.0,23.3,7.3
1100,25.6,0.255,0.489,92.0,11.4,32.9,46,11.4,7.9,23.3,...,26.4,46.8,36.6,38.1,25.2,62.6,45.8,26.5,23.3,4.5
1101,27.8,0.209,0.383,88.7,14.2,35.5,34,9.0,7.7,23.1,...,28.1,47.9,46.7,30.5,22.8,62.6,41.1,27.6,24.9,6.4
1102,21.0,0.258,0.429,90.6,11.2,30.3,37,8.4,7.3,24.1,...,24.1,47.5,37.4,35.1,27.6,62.8,44.4,23.7,24.4,7.5


In [203]:
#Assign Variables to be Trained
y = df_cleaned["batting_avg"]
X = df_cleaned.drop(columns=['batting_avg', 'k_percent'], inplace=False)

In [209]:
X

,slg_percent,exit_velocity_avg,launch_angle_avg,sweet_spot_percent,barrel,barrel_batted_rate,solidcontact_percent,flareburner_percent,poorlyunder_percent,poorlytopped_percent,...,whiff_percent,swing_percent,pull_percent,straightaway_percent,opposite_percent,f_strike_percent,groundballs_percent,flyballs_percent,linedrives_percent,popups_percent
0,0.409,88.5,10.8,28.5,21,5.0,5.2,21.6,24.5,39.0,...,23.1,53.4,34.7,40.6,24.5,64.0,49.4,22.6,20.9,7.1
1,0.553,93.0,15.7,34.8,58,13.1,8.4,24.9,24.0,27.1,...,23.2,44.7,41.9,36.4,21.7,52.1,37.6,25.6,28.7,8.1
2,0.486,91.3,12.2,31.4,42,10.9,8.1,20.3,23.9,35.1,...,32.0,43.9,38.4,43.9,16.9,60.3,43.6,24.9,24.9,6.5
3,0.423,87.4,15.8,33.5,23,5.6,6.6,23.3,30.3,29.4,...,17.9,52.9,36.9,38.3,24.8,65.5,37.6,24.5,29.1,8.7
4,0.453,89.5,12.6,35.7,28,5.5,7.8,26.3,23.5,33.7,...,16.8,48.1,35.3,41.4,23.3,61.9,42.4,18.4,31.6,7.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1099,0.506,90.0,11.0,32.7,34,7.6,7.8,22.0,22.7,34.2,...,21.5,45.8,38.4,36.9,24.7,57.9,45.3,24.0,23.3,7.3
1100,0.489,92.0,11.4,32.9,46,11.4,7.9,23.3,20.8,32.9,...,26.4,46.8,36.6,38.1,25.2,62.6,45.8,26.5,23.3,4.5
1101,0.383,88.7,14.2,35.5,34,9.0,7.7,23.1,23.9,31.3,...,28.1,47.9,46.7,30.5,22.8,62.6,41.1,27.6,24.9,6.4
1102,0.429,90.6,11.2,30.3,37,8.4,7.3,24.1,23.9,33.7,...,24.1,47.5,37.4,35.1,27.6,62.8,44.4,23.7,24.4,7.5


In [204]:
#Split the data into training & testing
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.2, random_state = 50)


In [205]:
#Create a Rabndom Forest Regressor Model
rf_model = RandomForestRegressor(n_estimators=100, random_state=50)

In [206]:
#Train the Model
rf_model.fit(X_train, y_train)

RandomForestRegressor(random_state=50)

In [207]:
#Make Predictions
predictions = rf_model.predict(X_test)
print(predictions)


[0.28972 0.29098 0.23092 0.28677 0.29703 0.25855 0.26573 0.27038 0.22962
 0.25739 0.30412 0.27225 0.27847 0.27005 0.25275 0.24375 0.27771 0.26777
 0.28567 0.27841 0.26256 0.25896 0.27285 0.23861 0.25099 0.30289 0.25154
 0.25583 0.25333 0.26083 0.27222 0.27499 0.24596 0.27946 0.26696 0.26547
 0.26017 0.23305 0.277   0.25801 0.26906 0.2516  0.26333 0.31781 0.27137
 0.26189 0.26535 0.29513 0.27154 0.25086 0.25195 0.27685 0.24032 0.24906
 0.28314 0.27612 0.2892  0.24108 0.24962 0.26687 0.29101 0.28858 0.28199
 0.27315 0.26283 0.28722 0.24554 0.24265 0.28928 0.24833 0.28137 0.23022
 0.24426 0.26344 0.25971 0.23097 0.26343 0.24528 0.23203 0.28383 0.2652
 0.24927 0.26739 0.26648 0.27682 0.27138 0.27852 0.25503 0.27057 0.28002
 0.27862 0.25738 0.26463 0.2336  0.24196 0.30079 0.25772 0.28325 0.31659
 0.26198 0.23595 0.26335 0.29879 0.30297 0.25701 0.25778 0.24237 0.26482
 0.26475 0.29802 0.28679 0.25422 0.31176 0.27268 0.25776 0.30821 0.25374
 0.22692 0.28322 0.27203 0.28283 0.27611 0.28979 0.2

In [208]:
mean = mean_absolute_error(y_test, predictions)
print("Mean", mean)

Mean 0.011428687782805435


In [199]:
importances = rf_model.feature_importances_
feature_names = X.columns
feature_importances = sorted(zip(importances, feature_names), reverse=True)
feature_importances

[(0.3400848046513085, 'k_percent'),
 (0.21026030040584714, 'barrel_batted_rate'),
 (0.0635759964745902, 'f_strike_percent'),
 (0.03295249953589342, 'whiff_percent'),
 (0.031037858160618076, 'exit_velocity_avg'),
 (0.02497021253573176, 'meatball_swing_percent'),
 (0.02371243047698067, 'oz_swing_percent'),
 (0.022806199960252117, 'sweet_spot_percent'),
 (0.02241913837648265, 'z_swing_miss_percent'),
 (0.020103243290733718, 'avg_hyper_speed'),
 (0.018547518638839595, 'flyballs_percent'),
 (0.01844901131771661, 'solidcontact_percent'),
 (0.017811660876717445, 'groundballs_percent'),
 (0.01728048813294086, 'pull_percent'),
 (0.015023369451881262, 'oz_contact_percent'),
 (0.013124624824328886, 'opposite_percent'),
 (0.012480749785914796, 'flareburner_percent'),
 (0.01065768623565184, 'swing_percent'),
 (0.00998503888387257, 'straightaway_percent'),
 (0.00987513158597199, 'poorlyunder_percent'),
 (0.008408668990883324, 'barrel'),
 (0.008220046336364665, 'avg_best_speed'),
 (0.0077433737855507

In [210]:
importances

(0.3400848046513085,
 0.21026030040584714,
 0.0635759964745902,
 0.03295249953589342,
 0.031037858160618076,
 0.02497021253573176,
 0.02371243047698067,
 0.022806199960252117,
 0.02241913837648265,
 0.020103243290733718,
 0.018547518638839595,
 0.01844901131771661,
 0.017811660876717445,
 0.01728048813294086,
 0.015023369451881262,
 0.013124624824328886,
 0.012480749785914796,
 0.01065768623565184,
 0.00998503888387257,
 0.00987513158597199,
 0.008408668990883324,
 0.008220046336364665,
 0.007743373785550793,
 0.007654695642083388,
 0.007278118089750949,
 0.005857406011679291,
 0.005454110455024997,
 0.00484762150098875,
 0.004689940177683562,
 0.004688055407716193)

In [202]:
importances, names = zip(*feature_importances)

# Create a bar chart
plt.barh(names, importances)
plt.xlabel('Importance')
plt.title('Feature Importances')
plt.show()

NameError: name 'plt' is not defined

In [184]:
X_test.columns

Index(['slg_percent', 'exit_velocity_avg', 'launch_angle_avg',
       'sweet_spot_percent', 'barrel', 'barrel_batted_rate',
       'solidcontact_percent', 'flareburner_percent', 'poorlyunder_percent',
       'poorlytopped_percent', 'poorlyweak_percent', 'hard_hit_percent',
       'avg_best_speed', 'avg_hyper_speed', 'z_swing_percent',
       'z_swing_miss_percent', 'oz_swing_percent', 'oz_swing_miss_percent',
       'oz_contact_percent', 'meatball_swing_percent', 'iz_contact_percent',
       'whiff_percent', 'swing_percent', 'pull_percent',
       'straightaway_percent', 'opposite_percent', 'f_strike_percent',
       'groundballs_percent', 'flyballs_percent', 'linedrives_percent',
       'popups_percent'],
      dtype='object')

In [185]:
# Close SQL connection
connection.close()

In [ ]:
from joblib import dump, load
dump(rf_model, 'baseball_stats.joblib')

['baseball_stats.joblib']